# Python Fortgeschritten: Eigene Ausnahmen
## Tag 2 - Notebook 10
***
In diesem Notebook wird behandelt:
- Custom Exceptions erstellen
- Exception-Hierarchien
- Best Practices
***


## 1 Eigene Exceptions

Wir können eigene Exception-Klassen erstellen, die von `Exception` erben.

### Warum eigene Exceptions?

Eigene Exceptions ermöglichen **spezifische Fehlerbehandlung** und verbessern die **Code-Organisation**. Statt generische Exceptions wie `ValueError` oder `RuntimeError` zu verwenden, können wir semantisch aussagekräftige Fehler definieren, die genau beschreiben, was schiefgelaufen ist. Das macht Code wartbarer und erleichtert das Debugging.

### Wann verwenden?

Eigene Exceptions sollten verwendet werden:
- **Bei domänenspezifischen Fehlern**: Wenn die Anwendung in einem speziellen Bereich arbeitet (z.B. Messgeräte, Netzwerk-Protokolle), können domänenspezifische Exceptions die Fehler besser beschreiben
- **Für Exception-Hierarchien**: Verschiedene Fehlertypen können in einer Hierarchie organisiert werden, um sowohl spezifische als auch allgemeine Behandlung zu ermöglichen
- **Für bessere Debugging-Informationen**: Eigene Exceptions können zusätzliche Informationen speichern (z.B. den fehlerhaften Wert, erlaubte Bereiche, Kontext)

### Vorteile

- **Semantisch aussagekräftige Fehler**: `TemperatureOutOfRangeError` ist klarer als `ValueError`
- **Bessere Wartbarkeit**: Code wird selbstdokumentierend durch aussagekräftige Exception-Namen
- **Spezifische Behandlung möglich**: Verschiedene Fehlertypen können unterschiedlich behandelt werden
- **Erweiterbarkeit**: Exception-Hierarchien ermöglichen sowohl spezifische als auch allgemeine Fehlerbehandlung


In [ ]:
class CustomError(Exception):
    pass

class ValueTooLargeError(Exception):
    def __init__(self, value, max_value):
        self.value = value
        self.max_value = max_value
        self.message = f"Wert {value} ist größer als Maximum {max_value}"
        super().__init__(self.message)

# Verwendung
try:
    value = 100
    if value > 50:
        raise ValueTooLargeError(value, 50)
except ValueTooLargeError as e:
    print(e)


#### 3.1 Aufgaben:


### Aufgabe (a): Messbereichs-Exception erstellen

Ein Temperatursensor hat einen definierten Messbereich von -50°C bis +150°C. Erstelle eine Exception-Klasse `MeasurementRangeError`, die ausgelöst wird, wenn ein Messwert außerhalb dieses Bereichs liegt.

**Anforderungen:**
- Erstelle eine Exception-Klasse `MeasurementRangeError`, die von `Exception` erbt
- Die Exception soll **drei Attribute** speichern:
  - `value`: Der gemessene Wert (der außerhalb des Bereichs liegt)
  - `min_value`: Minimaler erlaubter Wert
  - `max_value`: Maximaler erlaubter Wert
- Die Exception soll eine **Fehlermeldung** im Format haben: `f"Messwert {value} liegt außerhalb des erlaubten Bereichs [{min_value}, {max_value}]"`
- Verwende `super().__init__(message)` für korrekte Exception-Initialisierung
- Die Funktion `validate_temperature` soll die Exception werfen, wenn `value < min_temp` oder `value > max_temp`

**Erwartete Ausgaben:**
- `validate_temperature(25, -50, 150)` → `25` (funktioniert)
- `validate_temperature(-60, -50, 150)` → `MeasurementRangeError: "Messwert -60 liegt außerhalb des erlaubten Bereichs [-50, 150]"`
- `validate_temperature(200, -50, 150)` → `MeasurementRangeError: "Messwert 200 liegt außerhalb des erlaubten Bereichs [-50, 150]"`

**Hinweis:** Verwende `super().__init__()` für korrekte Exception-Initialisierung


In [ ]:
# TODO: Schritt 1 - Erstelle die Exception-Klasse MeasurementRangeError
#       class MeasurementRangeError(Exception):
#           def __init__(self, value, min_value, max_value):
#               # Speichere value, min_value, max_value als self.value, self.min_value, self.max_value
#               # Erstelle message = f"Messwert {value} liegt außerhalb des erlaubten Bereichs [{min_value}, {max_value}]"
#               # Rufe super().__init__(message) auf

def validate_temperature(value, min_temp=-50, max_temp=150):
    """
    Validiert einen Temperaturwert gegen den Messbereich.
    
    Args:
        value: Der zu prüfende Temperaturwert
        min_temp: Minimaler erlaubter Wert (Standard: -50)
        max_temp: Maximaler erlaubter Wert (Standard: 150)
    
    Returns:
        Der validierte Wert (wenn im Bereich)
    
    Raises:
        MeasurementRangeError: Wenn der Wert außerhalb des Bereichs liegt
    """
    # TODO: Schritt 2 - Prüfe, ob value < min_temp oder value > max_temp
    # TODO: Schritt 3 - Wenn außerhalb: raise MeasurementRangeError(value, min_temp, max_temp)
    # TODO: Schritt 4 - Wenn im Bereich: return value
    
    pass

# Teste mit:
try:
    print(validate_temperature(25))   # Erwartet: 25 (funktioniert)
    validate_temperature(-60)          # Erwartet: MeasurementRangeError
    validate_temperature(200)          # Erwartet: MeasurementRangeError
except MeasurementRangeError as e:
    print(f"Fehler: {e}")


### Aufgabe (b): Messgeräte-Exception-Hierarchie

Ein Messsystem besteht aus verschiedenen Komponenten (Sensoren, Kalibrierung, Datenübertragung). Erstelle eine Exception-Hierarchie.

**Anforderungen:**
- Erstelle eine **Basis-Exception** `MeasurementError`, die von `Exception` erbt
- Erstelle **drei Unterklassen**, die alle von `MeasurementError` erben:
  - `SensorError`: Für Sensor-spezifische Fehler
  - `CalibrationError`: Für Kalibrierungsfehler (z.B. wenn die Kalibrierung fehlschlägt, Kalibrierungsparameter ungültig sind, oder die Kalibrierungsdatei nicht gelesen werden kann)
  - `DataTransmissionError`: Für Datenübertragungsfehler
- Die Funktion `simulate_measurement_error` soll:
  - Bei `error_type == 'sensor'`: `SensorError("Sensor liefert keine Daten")` werfen
  - Bei `error_type == 'calibration'`: `CalibrationError("Kalibrierung fehlgeschlagen")` werfen
  - Bei `error_type == 'transmission'`: `DataTransmissionError("Datenübertragung unterbrochen")` werfen
  - Bei unbekanntem Typ: `MeasurementError(f"Unbekannter Fehlertyp: {error_type}")` werfen

**Vererbungsstruktur:**
```
Exception
  └── MeasurementError (Basis)
       ├── SensorError
       ├── CalibrationError
       └── DataTransmissionError
```

**Erwartete Ausgaben:**
- `simulate_measurement_error('sensor')` → `SensorError: "Sensor liefert keine Daten"`
- `simulate_measurement_error('calibration')` → `CalibrationError: "Kalibrierung fehlgeschlagen"`
- `simulate_measurement_error('transmission')` → `DataTransmissionError: "Datenübertragung unterbrochen"`

**Hinweis:** Exception-Hierarchien ermöglichen spezifische oder allgemeine Fehlerbehandlung


In [ ]:
# TODO: Schritt 1 - Erstelle die Exception-Hierarchie
#       class MeasurementError(Exception):
#           pass
#       
#       class SensorError(MeasurementError):
#           pass
#       
#       class CalibrationError(MeasurementError):
#           pass
#       
#       class DataTransmissionError(MeasurementError):
#           pass

def simulate_measurement_error(error_type):
    """
    Simuliert verschiedene Messfehler.
    
    Args:
        error_type: Art des Fehlers ('sensor', 'calibration', 'transmission')
    
    Raises:
        SensorError, CalibrationError, DataTransmissionError oder MeasurementError
    """
    # TODO: Schritt 2 - Prüfe error_type mit if/elif
    # TODO: Schritt 3 - Bei 'sensor': raise SensorError("Sensor liefert keine Daten")
    # TODO: Schritt 4 - Bei 'calibration': raise CalibrationError("Kalibrierung fehlgeschlagen")
    # TODO: Schritt 5 - Bei 'transmission': raise DataTransmissionError("Datenübertragung unterbrochen")
    # TODO: Schritt 6 - Bei unbekanntem Typ: raise MeasurementError(f"Unbekannter Fehlertyp: {error_type}")
    
    pass

# Teste mit:
try:
    simulate_measurement_error('sensor')
except MeasurementError as e:
    print(f"Messfehler gefangen: {e}")  # Erwartet: "Sensor liefert keine Daten"


### Aufgabe (c): Temperatur-Sensor-Validierung mit Generator

Ein Temperatursensor liefert kontinuierlich Daten in einem Datenstrom. Erstelle eine Exception-Klasse `InvalidTemperatureError` für ungültige Temperaturwerte und implementiere einen Generator zur Validierung.

**Anforderungen:**
- Die Exception-Klasse `InvalidTemperatureError` ist bereits vorhanden, erweitere sie:
  - Sie soll zwei Attribute speichern: `value` (der ungültige Wert) und `reason` (Grund für Ungültigkeit)
  - Fehlermeldung: `f"Ungültiger Temperaturwert {value}: {reason}"`
- Der Generator `temperature_data_stream` soll:
  - Über `raw_data` iterieren
  - **Validierungsregeln** anwenden:
    - Wenn Wert `None` ist → `raise InvalidTemperatureError(value, "Wert ist None")`
    - Wenn Wert nicht zu `float` konvertierbar ist → `raise InvalidTemperatureError(value, "Wert ist nicht numerisch")`
    - Wenn Wert < -273.15 (absoluter Nullpunkt) → `raise InvalidTemperatureError(temp, f"Unter absolutem Nullpunkt ({min_temp}°C)")`
    - Wenn Wert > 1000 (physikalisch unmöglich) → `raise InvalidTemperatureError(temp, f"Physikalisch unmöglich (> {max_temp}°C)")`
  - Bei gültigen Werten: `yield temp` (konvertierter float-Wert)

**Erwartete Ausgabe:**
- Bei `[20.5, None, ...]`: Generator wirft `InvalidTemperatureError` beim ersten `None`-Wert
- Bei `[20.5, -5.0, "invalid", ...]`: Generator wirft `InvalidTemperatureError` beim String "invalid"
- Bei `[20.5, -5.0, 25.3, -300, ...]`: Generator wirft `InvalidTemperatureError` bei -300 (unter absolutem Nullpunkt)

**Hinweis:** Generatoren (yield) sind nützlich für kontinuierliche Datenströme ([`Generatoren`](../1/00_iteratoren_generatoren.ipynb))


In [ ]:
class InvalidTemperatureError(Exception):
    """Exception für ungültige Temperaturwerte."""
    # TODO: Schritt 1 - Erweitere __init__ Methode:
    #       def __init__(self, value, reason):
    #           self.value = value
    #           self.reason = reason
    #           message = f"Ungültiger Temperaturwert {value}: {reason}"
    #           super().__init__(message)
    pass

def temperature_data_stream(raw_data):
    """
    Generator, der Temperaturdaten verarbeitet und validiert.
    
    Args:
        raw_data: Iterable mit rohen Temperaturwerten
    
    Yields:
        Gültige Temperaturwerte (float)
    
    Raises:
        InvalidTemperatureError: Bei ungültigen Werten (None, nicht-numerisch, außerhalb physikalischer Grenzen)
    """
    min_temp = -273.15  # Absoluter Nullpunkt
    max_temp = 1000     # Maximaler physikalisch sinnvoller Wert
    
    # TODO: Schritt 2 - Iteriere über raw_data mit for value in raw_data:
    # TODO: Schritt 3 - Prüfe, ob value None ist → raise InvalidTemperatureError(value, "Wert ist None")
    # TODO: Schritt 4 - Versuche, value zu float zu konvertieren (try/except)
    #       - Bei TypeError/ValueError: raise InvalidTemperatureError(value, "Wert ist nicht numerisch")
    # TODO: Schritt 5 - Prüfe, ob temp < min_temp → raise InvalidTemperatureError(temp, f"Unter absolutem Nullpunkt ({min_temp}°C)")
    # TODO: Schritt 6 - Prüfe, ob temp > max_temp → raise InvalidTemperatureError(temp, f"Physikalisch unmöglich (> {max_temp}°C)")
    # TODO: Schritt 7 - Wenn gültig: yield temp
    
    pass

# Teste mit:
test_data = [20.5, None, -5.0, "invalid", 25.3, -300, 30.1]
try:
    for temp in temperature_data_stream(test_data):
        print(f"Gültige Temperatur: {temp}°C")
except InvalidTemperatureError as e:
    print(f"Fehler: {e}")  # Erwartet: Fehler beim ersten ungültigen Wert (None)


### Aufgabe (d): Multi-Sensor-System mit Exception-Hierarchie

Ein Multi-Sensor-System überwacht verschiedene physikalische Größen (Temperatur, Druck, Feuchtigkeit). Erstelle eine umfassende Exception-Hierarchie.

**Anforderungen:**
- Erstelle eine **Exception-Hierarchie** mit folgender Struktur:
  - Basis: `MeasurementSystemError` (erbt von `Exception`)
  - `SensorError` (erbt von `MeasurementSystemError`) mit drei Unterklassen:
    - `TemperatureSensorError` (erbt von `SensorError`)
    - `PressureSensorError` (erbt von `SensorError`)
    - `HumiditySensorError` (erbt von `SensorError`)
  - `SystemError` (erbt von `MeasurementSystemError`) für System-weite Fehler

**Vererbungsstruktur:**
```
Exception
  └── MeasurementSystemError
       ├── SensorError
       │    ├── TemperatureSensorError
       │    ├── PressureSensorError
       │    └── HumiditySensorError
       └── SystemError
```

- Die Funktion `process_sensor_data` soll:
  - Ein **Dictionary** mit Sensor-Konfigurationen erstellen:
    - `'temperature'`: min=-50, max=150, error=TemperatureSensorError
    - `'pressure'`: min=0, max=1000, error=PressureSensorError
    - `'humidity'`: min=0, max=100, error=HumiditySensorError
  - Wenn `sensor_type` nicht im Dictionary ist → `raise SystemError(f"Unbekannter Sensor-Typ: {sensor_type}")`
  - Wenn `value < min` oder `value > max` → `raise ErrorClass(f"{sensor_type.capitalize()}-Wert {value} außerhalb des Bereichs [{min_val}, {max_val}]")`
  - Bei gültigem Wert: `return value`

**Erwartete Ausgaben:**
- `process_sensor_data('temperature', 200)` → `TemperatureSensorError: "Temperature-Wert 200 außerhalb des Bereichs [-50, 150]"`
- `process_sensor_data('pressure', -5)` → `PressureSensorError: "Pressure-Wert -5 außerhalb des Bereichs [0, 1000]"`
- `process_sensor_data('humidity', 150)` → `HumiditySensorError: "Humidity-Wert 150 außerhalb des Bereichs [0, 100]"`

**Hinweis:** Dict Comprehension kann verwendet werden, um Sensor-Mappings zu erstellen ([`Dict Comprehensions`](../1/01_comprehensions.ipynb))


In [ ]:
# TODO: Schritt 1 - Erstelle die Exception-Hierarchie:
#       class MeasurementSystemError(Exception):
#           pass
#       
#       class SensorError(MeasurementSystemError):
#           pass
#       
#       class TemperatureSensorError(SensorError):
#           pass
#       
#       class PressureSensorError(SensorError):
#           pass
#       
#       class HumiditySensorError(SensorError):
#           pass
#       
#       class SystemError(MeasurementSystemError):
#           pass

def process_sensor_data(sensor_type, value):
    """
    Verarbeitet Sensordaten und wirft spezifische Exceptions bei Fehlern.
    
    Args:
        sensor_type: Art des Sensors ('temperature', 'pressure', 'humidity')
        value: Der Messwert
    
    Returns:
        Validierter Wert (wenn im gültigen Bereich)
    
    Raises:
        TemperatureSensorError, PressureSensorError, HumiditySensorError oder SystemError
    """
    # TODO: Schritt 2 - Erstelle Dictionary mit Sensor-Konfigurationen:
    #       sensor_ranges = {
    #           'temperature': {'min': -50, 'max': 150, 'error': TemperatureSensorError},
    #           'pressure': {'min': 0, 'max': 1000, 'error': PressureSensorError},
    #           'humidity': {'min': 0, 'max': 100, 'error': HumiditySensorError}
    #       }
    # TODO: Schritt 3 - Prüfe, ob sensor_type in sensor_ranges ist
    #       - Wenn nicht: raise SystemError(f"Unbekannter Sensor-Typ: {sensor_type}")
    # TODO: Schritt 4 - Hole min_val, max_val, ErrorClass aus sensor_ranges[sensor_type]
    # TODO: Schritt 5 - Prüfe, ob value < min_val oder value > max_val
    #       - Wenn ja: raise ErrorClass(f"{sensor_type.capitalize()}-Wert {value} außerhalb des Bereichs [{min_val}, {max_val}]")
    # TODO: Schritt 6 - Wenn gültig: return value
    
    pass

# Teste mit:
sensors = {
    'temperature': 200,  # Zu hoch
    'pressure': -5,      # Negativ (ungültig)
    'humidity': 150      # Zu hoch (>100%)
}

for sensor_type, value in sensors.items():
    try:
        result = process_sensor_data(sensor_type, value)
        print(f"{sensor_type}: {result}")
    except MeasurementSystemError as e:
        print(f"Fehler bei {sensor_type}: {e}")  # Erwartet: Spezifische Sensor-Exceptions


#### Lösung:


In [ ]:
# Musterlösung (a): Messbereichs-Exception
class MeasurementRangeError(Exception):
    """Exception für Werte außerhalb des Messbereichs."""
    def __init__(self, value, min_value, max_value):
        self.value = value
        self.min_value = min_value
        self.max_value = max_value
        message = f"Messwert {value} liegt außerhalb des erlaubten Bereichs [{min_value}, {max_value}]"
        super().__init__(message)

def validate_temperature(value, min_temp=-50, max_temp=150):
    """
    Validiert einen Temperaturwert gegen den Messbereich.
    """
    if value < min_temp or value > max_temp:
        raise MeasurementRangeError(value, min_temp, max_temp)
    return value

# Teste mit:
try:
    print(validate_temperature(25))   # Funktioniert
    validate_temperature(-60)          # Exception
except MeasurementRangeError as e:
    print(f"Fehler: {e}")

# Musterlösung (b): Messgeräte-Exception-Hierarchie
class MeasurementError(Exception):
    """Basis-Exception für alle Messfehler."""
    pass

class SensorError(MeasurementError):
    """Exception für Sensor-spezifische Fehler."""
    pass

class CalibrationError(MeasurementError):
    """Exception für Kalibrierungsfehler."""
    pass

class DataTransmissionError(MeasurementError):
    """Exception für Datenübertragungsfehler."""
    pass

def simulate_measurement_error(error_type):
    """
    Simuliert verschiedene Messfehler.
    """
    if error_type == 'sensor':
        raise SensorError("Sensor liefert keine Daten")
    elif error_type == 'calibration':
        raise CalibrationError("Kalibrierung fehlgeschlagen")
    elif error_type == 'transmission':
        raise DataTransmissionError("Datenübertragung unterbrochen")
    else:
        raise MeasurementError(f"Unbekannter Fehlertyp: {error_type}")

# Teste mit:
try:
    simulate_measurement_error('sensor')
except MeasurementError as e:
    print(f"Messfehler gefangen: {e}")


# Musterlösung (c): Temperatur-Sensor-Validierung mit Generator
class InvalidTemperatureError(Exception):
    """Exception für ungültige Temperaturwerte."""
    def __init__(self, value, reason):
        self.value = value
        self.reason = reason
        super().__init__(f"Ungültiger Temperaturwert {value}: {reason}")

def temperature_data_stream(raw_data):
    """
    Generator, der Temperaturdaten verarbeitet und validiert.
    """
    min_temp = -273.15  # Absoluter Nullpunkt
    max_temp = 1000     # Maximaler physikalisch sinnvoller Wert
    
    for value in raw_data:
        # Prüfe auf None
        if value is None:
            raise InvalidTemperatureError(value, "Wert ist None")
        
        # Prüfe auf numerischen Typ
        try:
            temp = float(value)
        except (TypeError, ValueError):
            raise InvalidTemperatureError(value, "Wert ist nicht numerisch")
        
        # Prüfe auf physikalisch möglichen Bereich
        if temp < min_temp:
            raise InvalidTemperatureError(temp, f"Unter absolutem Nullpunkt ({min_temp}°C)")
        if temp > max_temp:
            raise InvalidTemperatureError(temp, f"Physikalisch unmöglich (> {max_temp}°C)")
        
        # Yield gültigen Wert
        yield temp

# Teste mit:
test_data = [20.5, None, -5.0, "invalid", 25.3, -300, 30.1]
try:
    for temp in temperature_data_stream(test_data):
        print(f"Gültige Temperatur: {temp}°C")
except InvalidTemperatureError as e:
    print(f"Fehler: {e}")


# Musterlösung (d): Multi-Sensor-System
class MeasurementSystemError(Exception):
    """Basis-Exception für Messsystem-Fehler."""
    pass

class SensorError(MeasurementSystemError):
    """Basis-Exception für Sensor-Fehler."""
    pass

class TemperatureSensorError(SensorError):
    """Exception für Temperatursensor-Fehler."""
    pass

class PressureSensorError(SensorError):
    """Exception für Drucksensor-Fehler."""
    pass

class HumiditySensorError(SensorError):
    """Exception für Feuchtigkeitssensor-Fehler."""
    pass

class SystemError(MeasurementSystemError):
    """Exception für System-weite Fehler."""
    pass

def process_sensor_data(sensor_type, value):
    """
    Verarbeitet Sensordaten und wirft spezifische Exceptions bei Fehlern.
    """
    # Validierungsbereiche für verschiedene Sensoren
    # Dict Comprehension für Sensor-Mapping
    sensor_ranges = {
        'temperature': {'min': -50, 'max': 150, 'error': TemperatureSensorError},
        'pressure': {'min': 0, 'max': 1000, 'error': PressureSensorError},
        'humidity': {'min': 0, 'max': 100, 'error': HumiditySensorError}
    }
    
    # Prüfe ob Sensor-Typ bekannt ist
    if sensor_type not in sensor_ranges:
        raise SystemError(f"Unbekannter Sensor-Typ: {sensor_type}")
    
    # Hole Validierungsparameter
    sensor_config = sensor_ranges[sensor_type]
    min_val = sensor_config['min']
    max_val = sensor_config['max']
    ErrorClass = sensor_config['error']
    
    # Validiere Wert
    if value < min_val or value > max_val:
        raise ErrorClass(
            f"{sensor_type.capitalize()}-Wert {value} außerhalb des Bereichs [{min_val}, {max_val}]"
        )
    
    return value

# Teste mit:
sensors = {
    'temperature': 200,  # Zu hoch
    'pressure': -5,      # Negativ (ungültig)
    'humidity': 150      # Zu hoch (>100%)
}

for sensor_type, value in sensors.items():
    try:
        result = process_sensor_data(sensor_type, value)
        print(f"{sensor_type}: {result}")
    except MeasurementSystemError as e:
        print(f"Fehler bei {sensor_type}: {e}")

